In [48]:
import requests
import json
import pandas as pd

url = "https://www.weather.gov/source/aprfc/it_2022.json"

response = requests.get(url)
data = json.loads(response.text)

# Make sure that the data is a dictionary
if isinstance(data, dict):
    # Get the features from the JSON data
    months = data.values()

    # Iterate through the features
    for month in months:
        
        # Do something with the feature
        print(month)
else:
    # Handle the case where the data is not a dictionary
    print("Error: JSON data is not a dictionary")

{'02': {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-150.503, 62.1925]}, 'metadata': {'agency': 'NWS', 'created': 1682000947000}, 'properties': {'waterYr': 2021, 'lid': 'ABRA2', 'name': 'Amber Lake North', 'elev': '449', 'averages': {'01': {'avg': 17, 'num': 13}, '02': {'avg': 23, 'num': 13}, '03': {'avg': 27, 'num': 10}, '04': {'avg': 30, 'num': 7}, '05': {'avg': 25, 'num': 3}, '11': {'avg': 3, 'num': 2}, '12': {'avg': 10, 'num': 9}}, 'data': [['2022-02-02 18:00:00', '24']]}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-146.666, 64.3047]}, 'metadata': {'agency': 'NWS', 'created': 1682000947000}, 'properties': {'waterYr': 2021, 'lid': 'BRLA2', 'name': 'Birch Lake Ice', 'elev': '700', 'averages': {'01': {'avg': 23, 'num': 23}, '02': {'avg': 30, 'num': 22}, '03': {'avg': 36, 'num': 21}, '04': {'avg': 37, 'num': 22}, '05': {'avg': 37, 'num': 1}, '11': {'avg': 6, 'num': 10}, '12': {'avg': 15, 'num': 24}}

In [3]:
print(data['months']['01']['features'][2]['properties']['lid'])

CPIA2


In [60]:
rows=[]

#Need to iterate through the various levels of the json

for items in data:
    print(items)
    for layer1 in data[items]:
        print("###################")
        print("  ",layer1)
        
       # for layer2 in data[items][layer1]:
    #    print("--------------------------------")
       #     print("  ",layer2)
            
        for layer3 in data[items][layer1]["features"]:

            #determine how many measurements were taken at each location
            numOfMeasurements = len(layer3['properties']['data'])


            date_and_measure_list = (layer3['properties']['data'])
            lid = layer3['properties']['lid']
            st_name = layer3['properties']['name']




            print(lid)
            print(st_name)
            print(layer3['properties']['data'])
            print(numOfMeasurements)
            print(date_and_measure_list)


            print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
            print("     ")


            #need to create a for loop here that creates multiple rows if there are more than one date/measurement
            n = (numOfMeasurements)
            i=0
            while i < n:
                rows.append([lid,st_name,date_and_measure_list[i][0],date_and_measure_list[i][1]])
                #print(rows)
                i+=1
                    
print(rows)                   
               


months
###################
   02
ABRA2
Amber Lake North
[['2022-02-02 18:00:00', '24']]
1
[['2022-02-02 18:00:00', '24']]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
     
BRLA2
Birch Lake Ice
[['2022-02-01 18:00:00', '24']]
1
[['2022-02-01 18:00:00', '24']]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
     
CHFA2
Chena River at Fairbanks
[['2022-02-01 21:59:00', '14']]
1
[['2022-02-01 21:59:00', '14']]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
     
CJXA2
Kuskokwim River at Crooked Creek
[['2022-02-01 20:00:00', '46']]
1
[['2022-02-01 20:00:00', '46']]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
     
CNPA2
Chena Lakes Ice
[['2022-02-01 19:18:00', '27']]
1
[['2022-02-01 19:18:00', '27']]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
     
CPIA2
Chena River at Pikes Landing Ice
[['2022-02-01 22:59:00', '24']]
1
[['2022-02-01 22:59:00', '24']]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
     
CTLA2
Cottonwood Lake
[['2022-02-01 00:00:00', '26']]
1
[['2

In [61]:
df=pd.DataFrame(rows, columns=["Lid","Name","Date","Measurement"])
pd.set_option('display.width',1000)
print(df)

       Lid                                   Name                 Date Measurement
0    ABRA2                       Amber Lake North  2022-02-02 18:00:00          24
1    BRLA2                         Birch Lake Ice  2022-02-01 18:00:00          24
2    CHFA2               Chena River at Fairbanks  2022-02-01 21:59:00          14
3    CJXA2       Kuskokwim River at Crooked Creek  2022-02-01 20:00:00          46
4    CNPA2                        Chena Lakes Ice  2022-02-01 19:18:00          27
5    CPIA2       Chena River at Pikes Landing Ice  2022-02-01 22:59:00          24
6    CTLA2                        Cottonwood Lake  2022-02-01 00:00:00          26
7    GRSA2             Gulkana River at Sourdough  2022-02-10 20:00:00          20
8    GYRA2                  Yukon River at Galena  2022-02-04 18:00:00          36
9    KRIA2           Kuskokwim River at Aniak Ice  2022-02-04 21:00:00          53
10   KZBA2                      Kotzebue Lake Ice  2022-02-02 21:00:00          32
11  

In [145]:
#df_expanded = df.assign(data_dict=df['data_dict'].apply(pd.Series)).explode('data_dict').reset_index(drop=True)